In [ ]:
#!pip install modin[ray]
#!pip install -U ipywidgets

In [1]:
import requests
from xml.etree import ElementTree as ET
#import pandas as pd
import modin.pandas as pd
import os
import concurrent.futures

In [2]:
os.makedirs("raw_data", exist_ok=True)

In [3]:
endpoint = 'https://storage.googleapis.com/validaciones_tmsa/'
def descargar_archivo(carpeta_destino, destino, url = endpoint):
    file = os.path.join(carpeta_destino, destino.split('/')[-1])
    print(f'Descargando {file}...')
    response_headers = requests.head(url+destino)
    print(f'Tamaño del archivo: {response_headers.headers["Content-Length"]} bytes \n estatus: {response_headers.status_code}')
    response = requests.get(url+destino)
    print(f'Estado de la descarga: {response.status_code}')
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
        print(f'Archivo descargado a {file}.')
    else:
        print(f'Error al descargar el archivo: {response.status_code}')

def descargar_archivos_paralelo(carpeta_destino, lista_destinos, url = endpoint):
    print(f'Descargando {len(lista_destinos)} archivos en paralelo...')
    with concurrent.futures.ThreadPoolExecutor() as executor:
        print(f'Usando {executor._max_workers} hilos...')
        executor.map(lambda x: descargar_archivo(carpeta_destino, x, url), lista_destinos)

In [103]:
def procesar_df(df):
    columns = ['Fecha_Transaccion', 'Dispositivo', 'Estacion_Parada']
    columns_upper = map(str.upper, columns)

    df = df[columns]
    df.columns = columns_upper

    df = df[df['ESTACION_PARADA'].str.contains('U. NACIONAL', na=False)].drop('ESTACION_PARADA', axis=1).convert_dtypes()
    df['DISPOSITIVO'] = df['DISPOSITIVO'].astype('int32')
    df['FECHA_TRANSACCION'] = pd.to_datetime(df['FECHA_TRANSACCION'], format='%Y-%m-%d %H:%M:%S')
    df['HORA'] = df['FECHA_TRANSACCION'].dt.time
    df['FECHA_TRANSACCION'] = df['FECHA_TRANSACCION'].dt.date
    df.sort_values(by=['FECHA_TRANSACCION', 'HORA'], ascending=[True, True], inplace=True)
    df = df[df['HORA'].between(pd.to_datetime('16:00:00').time(), pd.to_datetime('18:30:00').time())]
    df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S')
    df['DIFERENCIA'] = df.groupby(['FECHA_TRANSACCION', 'DISPOSITIVO'])['HORA'].diff().dt.total_seconds().fillna(0)
    df.sort_values(by=['FECHA_TRANSACCION', 'DISPOSITIVO', 'HORA'], ascending=[True, True, True], inplace=True)
    df['HORA'] = df['HORA'].dt.time
    df.reset_index(drop=True, inplace=True)

    return df

In [96]:
def procesar_df_lista(carpeta, file):
    f_name = os.path.join(carpeta, file.split('/')[1])

    try:
        df_tmp = pd.read_csv(f_name)
        df_tmp = procesar_df(df_tmp)
        print(f'Data frame {f_name} procesado')
        try:
            os.remove(f_name)
            print(f'Archivo {f_name} eliminado')
        except Exception as e:
            print(f'Error eliminando el archivo {f_name}: {e}')
            return None
    except Exception as e:
        print(f'Error procesando el df {f_name}: {e}')
        return None
    return df_tmp

In [97]:
response = requests.get(endpoint)
xml_data = response.text

root = ET.fromstring(xml_data)

In [98]:
files = []

namespace = {'ns': 'http://doc.s3.amazonaws.com/2006-03-01'}
total_size = 0
for element, size in zip(root.iterfind('.//ns:Key', namespace), root.iterfind('.//ns:Size', namespace)):
    if 'validacionTroncal2024' in element.text:
        files.append(element.text)
        total_size += int(size.text)

In [99]:
total_size_gb = total_size / (1024**3)
print(f'Tamaño total: {total_size_gb:.2f} GB')

Tamaño total: 44.95 GB


In [73]:
len(files)

86

In [28]:
import shutil

total, used, free = shutil.disk_usage("/")
free_gb = free / (1024**3)

print(f"Espacio libre en el disco duro: {free_gb:.2f} GB")


Espacio libre en el disco duro: 14.61 GB


In [ ]:
ingresos_ultimos_20d_2024= pd.DataFrame()

files_to_download = files[:-20]
length = len(files)
batches = 6
for i in range(batches):
    print(f'Procesando batch {i+1}/{batches}...')
    files_batch = files_to_download[i*length//batches:(i+1)*length//batches]
    descargar_archivos_paralelo("raw_data", files_batch)
    dataframes = [procesar_df_lista("raw_data", f) for f in files_batch]
    ingresos_ultimos_20d_2024 = pd.concat([ingresos_ultimos_20d_2024] + dataframes, join='outer', ignore_index=True)

print('**********TODOS LOS ARCHIVOS HAN SIDO PROCESADOS**********')

In [30]:
ingresos_ultimos_20d_2024.to_csv('ingresos_ultimos_20d_2024.csv')

In [87]:
ingresos_ultimos_20d_2024 = pd.read_csv('ingresos_ultimos_20d_2024.csv')